In [2]:
import csv
import numpy as np
import opendatasets as od
import pandas as pd
import os

In [3]:
od.download("https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/download?datasetVersionNumber=3")

Skipping, found downloaded files in ".\creditcardfraud" (use force=True to force download)


In [6]:
fname = "./creditcardfraud/creditcard.csv"
all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [9]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


<h6>Analyze class imbalance in the targets</h6>

In [10]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


<h6>Normalize the data using training set statistics</h6>

In [11]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

<h6>Build a binary classification model</h6>

In [12]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 139,777
Trainable params: 139,777
Non-trai

<h6>Train the model with class_weight argument</h6>

In [13]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 4s - loss: 2.2066e-06 - fn: 50.0000 - fp: 17335.0000 - tn: 210094.0000 - tp: 367.0000 - precision: 0.0207 - recall: 0.8801 - val_loss: 0.1410 - val_fn: 9.0000 - val_fp: 2266.0000 - val_tn: 54620.0000 - val_tp: 66.0000 - val_precision: 0.0283 - val_recall: 0.8800 - 4s/epoch - 34ms/step
Epoch 2/30
112/112 - 3s - loss: 1.5115e-06 - fn: 36.0000 - fp: 9133.0000 - tn: 218296.0000 - tp: 381.0000 - precision: 0.0400 - recall: 0.9137 - val_loss: 0.2344 - val_fn: 5.0000 - val_fp: 4991.0000 - val_tn: 51895.0000 - val_tp: 70.0000 - val_precision: 0.0138 - val_recall: 0.9333 - 3s/epoch - 24ms/step
Epoch 3/30
112/112 - 3s - loss: 1.2110e-06 - fn: 31.0000 - fp: 7345.0000 - tn: 220084.0000 - tp: 386.0000 - precision: 0.0499 - recall: 0.9257 - val_loss: 0.0810 - val_fn: 9.0000 - val_fp: 1323.0000 - val_tn: 55563.0000 - val_tp: 66.0000 - val_precision: 0.0475 - val_recall: 0.8800 - 3s/epoch - 25ms/step
Epoch 4/30
112/112 - 3s - loss: 1.0375e-06 - fn: 27.0000 - fp: 6470.0000 - tn: 22

Epoch 28/30
112/112 - 3s - loss: 2.1008e-07 - fn: 1.0000 - fp: 2763.0000 - tn: 224666.0000 - tp: 416.0000 - precision: 0.1309 - recall: 0.9976 - val_loss: 0.0106 - val_fn: 11.0000 - val_fp: 175.0000 - val_tn: 56711.0000 - val_tp: 64.0000 - val_precision: 0.2678 - val_recall: 0.8533 - 3s/epoch - 24ms/step
Epoch 29/30
112/112 - 3s - loss: 3.1205e-07 - fn: 4.0000 - fp: 3333.0000 - tn: 224096.0000 - tp: 413.0000 - precision: 0.1103 - recall: 0.9904 - val_loss: 0.0337 - val_fn: 9.0000 - val_fp: 421.0000 - val_tn: 56465.0000 - val_tp: 66.0000 - val_precision: 0.1355 - val_recall: 0.8800 - 3s/epoch - 23ms/step
Epoch 30/30
112/112 - 2s - loss: 1.6464e-07 - fn: 0.0000e+00 - fp: 1931.0000 - tn: 225498.0000 - tp: 417.0000 - precision: 0.1776 - recall: 1.0000 - val_loss: 0.0086 - val_fn: 11.0000 - val_fp: 96.0000 - val_tn: 56790.0000 - val_tp: 64.0000 - val_precision: 0.4000 - val_recall: 0.8533 - 2s/epoch - 22ms/step
